In [1]:
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate

# 使用本地 LLaMA 模型（例如 llama3）
llm = ChatOllama(model="llama3:8b", temperature=0.7)

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 定義輸出結構
class MessageClassification(BaseModel):
    verdict: str = Field(description="Verdict whether the message is Real or Fake")
    confidence: str = Field(description="Confidence level of the judgment (e.g., High, Medium, Low)")
    reason: str = Field(description="Brief explanation of the judgment")

# 建立解析器
parser = JsonOutputParser(pydantic_object=MessageClassification)

In [3]:
from langchain.prompts import PromptTemplate

# prompt + 格式指示語
format_instructions = parser.get_format_instructions()

prompt_template = PromptTemplate.from_template(
    """
You are a professional fact-checker. Analyze the following message and determine if it is real or fake.

Message:
\"\"\"{message}\"\"\"

{format_instructions}
"""
)

In [4]:
from langchain_core.runnables import RunnableSequence

# 建立推論流程
chain = (
    prompt_template | llm | parser
)

In [ ]:
message = "Breaking: NASA confirms aliens landed in New York. #UFO"
result = chain.invoke({
    "message": message,
    "format_instructions": parser.get_format_instructions()
})
print(result)

{'properties': {'verdict': 'Fake', 'confidence': 'High', 'reason': 'NASA has never confirmed the landing of aliens in New York or anywhere else. While there have been reported UFO sightings and alleged alien encounters, there is no credible evidence to support the claim that aliens have landed on Earth.'}}
